# Notes

In [1]:
import pandas as pd
from constants import *

In [2]:
from data_sources.data_from_yearly_shapefiles import DataFromYearlyShapefiles

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


/home/minigonche/Dropbox/Projects/pathogen_study_regions_generator/env/lib/python3.8/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [3]:
ds = DataFromYearlyShapefiles('COCA',
                        "coca",
                            "coca_fields_shapes",                            
                            "coca_fields_{year}.shp",
                            ['coca'],
                            2000,
                            2002,
                            normalize=None,
                            included_groupings=[TOTAL])

In [4]:
import geopandas 

# Loads the municipalities
df_geo = geopandas.read_file("/home/minigonche/Dropbox/Projects/malaria_mining/data/cleaned/geo/municipalities/municipalities.shp")
df_geo = df_geo.rename(columns = {'muni_id':ID})

In [5]:
df = ds.createData(df_geo, WEEK)
df

      Processing year: 2000
      Processing year: 2001
         No file found for year 2001
      Processing year: 2002


,ID,date,coca_total
0,5004,2000-01-02,6.519325e+07
1,5004,2000-01-09,6.519325e+07
2,5004,2000-01-16,6.519325e+07
3,5004,2000-01-23,6.519325e+07
4,5004,2000-01-30,6.519325e+07
...,...,...,...
28150,99773,2002-12-01,2.863384e+11
28151,99773,2002-12-08,2.863384e+11
28152,99773,2002-12-15,2.863384e+11
28153,99773,2002-12-22,2.863384e+11
